In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 30 20:30:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install octis

     |████████████████████████████████| 129 kB 8.6 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 24.1 MB 18 kB/s 
     |████████████████████████████████| 170 kB 69.9 MB/s 
     |████████████████████████████████| 22.3 MB 72.7 MB/s 
     |████████████████████████████████| 16.3 MB 42.7 MB/s 
     |████████████████████████████████| 100 kB 12.2 MB/s 
     |████████████████████████████████| 3.8 MB 92.1 MB/s 
     |████████████████████████████████| 1.2 MB 66.4 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 6.5 MB 49.5 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp37-cp37m-linux_x86_64.whl size=233363 sha256=bb4a2b2e4d3d4bd91ca965be1f54ebd9db6826a2d547b835deeb1705c332a2f9
  Stored in directory: /root/.cache/pip/wheels/cd/e8/1e/bf95cf256e4d3ffc94289ab508c49

#IMPORT


In [ ]:
import csv
import random
import os
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import trange

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/ProdLDA/

'Copia di ProdLDA.ipynb'
 HDFS_data_folder_MESSAGE
 HDFS_data_folder_ORIGIN+MESSAGE
 HDFS_data_folder_SPLIT
'HDFS_ProdLDA_Train_History_MESSAGE_nl=1.csv'
'HDFS_ProdLDA_Train_History_MESSAGE_nl=2.csv'
'HDFS_ProdLDA_Train_History_ORIGIN+MESSAGE_nl=1.csv'
'HDFS_ProdLDA_Train_History_ORIGIN+MESSAGE_nl=2.csv'
'HDFS_ProdLDA_Train_History_SPLIT_nl=1.csv'
'HDFS_ProdLDA_Train_History_SPLIT_nl=2.csv'
 HDFS_RESULTS_PRODLDA_MESSAGE.csv
 HDFS_RESULTS_PRODLDA_ORIGIN+MESSAGE.csv
 HDFS_RESULTS_PRODLDA_SPLIT.csv
 Input
 OLD_RESULT
 ProdLDA.ipynb
 Spark_data_folder_MESSAGE
 Spark_data_folder_ORIGIN+MESSAGE
 Spark_data_folder_SPLIT
'Spark_ProdLDA_Train_History_MESSAGE_nl=1.csv'
'Spark_ProdLDA_Train_History_MESSAGE_nl=2.csv'
'Spark_ProdLDA_Train_History_ORIGIN+MESSAGE_nl=1.csv'
'Spark_ProdLDA_Train_History_ORIGIN+MESSAGE_nl=2.csv'
 Spark_ProdLDA_Train_History_SPLIT.csv
'Spark_ProdLDA_Train_History_SPLIT_nl=1.csv'
'Spark_ProdLDA_Train_History_SPLIT_nl=2.csv'
 Spark_RESULTS_PRODLDA_MESSAGE.csv
 Spark_RESULT

In [ ]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/ProdLDA/

/content/gdrive/My Drive/Colab Notebooks/ProdLDA


#DATA

In [ ]:
pd.set_option('expand_frame_repr', False)

In [ ]:
pathname="/content/gdrive/MyDrive/Colab Notebooks/ProdLDA"

In [ ]:
name="Spark"
tipo="MESSAGE"

In [ ]:
df_log1=pd.read_csv(pathname+f"/{name}_ProdLDA_Train_History_{tipo}_nl=1.csv")

In [ ]:
print(df_log1.head(5))
print(df_log1.tail(5))

   Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum     lr     k  coherence  coherence15  coherence10  delta_k
0           0        1.0       100.0      0.1       100.0       0.8  0.001  17.0   0.653891     0.599194     0.539541      0.0
1           1        1.0       100.0      0.1       100.0       0.8  0.001  18.0   0.667492     0.624094     0.548601      0.0
2           2        1.0       100.0      0.1       100.0       0.8  0.001  19.0   0.668434     0.627315     0.565014      0.0
3           3        1.0       100.0      0.1       100.0       0.8  0.001  20.0   0.660124     0.612738     0.557246      0.0
4           4        1.0       100.0      0.1       100.0       0.8  0.001  21.0   0.660634     0.603351     0.525294      0.0
      Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum   lr     k  coherence  coherence15  coherence10  delta_k
5611        5611        1.0       200.0      0.3       200.0      0.99  0.5  38.0   0.463058     0.529946     

In [ ]:
df_log2=pd.read_csv(pathname+f"/{name}_ProdLDA_Train_History_{tipo}_nl=2.csv")

In [ ]:
print(df_log2.head(5))
print(df_log2.tail(5))

   Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum     lr     k  coherence  coherence15  coherence10  delta_k
0           0        2.0       100.0      0.1       100.0       0.8  0.001  17.0   0.706498     0.651262     0.572415      0.0
1           1        2.0       100.0      0.1       100.0       0.8  0.001  18.0   0.720877     0.678504     0.599862      0.0
2           2        2.0       100.0      0.1       100.0       0.8  0.001  19.0   0.710820     0.675800     0.581069      0.0
3           3        2.0       100.0      0.1       100.0       0.8  0.001  20.0   0.719591     0.685263     0.598646      0.0
4           4        2.0       100.0      0.1       100.0       0.8  0.001  21.0   0.734633     0.694061     0.615116      0.0
      Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum   lr     k  coherence  coherence15  coherence10  delta_k
5611        5611        2.0       200.0      0.3       200.0      0.99  0.5  38.0   0.467100     0.451884     

In [ ]:
df_log=pd.concat([df_log1, df_log2])

In [ ]:
print(df_log1.shape, df_log2.shape, df_log.shape)

(5616, 12) (5616, 12) (11232, 12)


In [ ]:
final_df0=df_log
final_df0=final_df0.sort_values(by=['coherence'], ascending=False)
print(len(final_df0))
print(final_df0.head(30))

11232
      Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum     lr     k  coherence  coherence15  coherence10  delta_k
5002        5002        2.0       200.0      0.3       100.0      0.99  0.001  27.0   0.750322     0.695560     0.594135      0.0
4846        4846        2.0       200.0      0.3       100.0      0.90  0.001  27.0   0.748818     0.697751     0.602929      0.0
4854        4854        2.0       200.0      0.3       100.0      0.90  0.001  35.0   0.748520     0.707640     0.623501      0.0
4066        4066        2.0       200.0      0.2       100.0      0.99  0.001  27.0   0.747545     0.696917     0.588816      0.0
5010        5010        2.0       200.0      0.3       100.0      0.99  0.001  35.0   0.747417     0.692161     0.599719      0.0
3918        3918        2.0       200.0      0.2       100.0      0.90  0.001  35.0   0.747229     0.695494     0.626002      0.0
4698        4698        2.0       200.0      0.3       100.0      0.80  0.001  35.0 

In [ ]:
final_df0=df_log[df_log['delta_k']==0]
final_df0=final_df0.sort_values(by=['coherence'], ascending=False)
print(len(final_df0))
print(final_df0.head(30))

2645
      Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum     lr     k  coherence  coherence15  coherence10  delta_k
5002        5002        2.0       200.0      0.3       100.0      0.99  0.001  27.0   0.750322     0.695560     0.594135      0.0
4846        4846        2.0       200.0      0.3       100.0      0.90  0.001  27.0   0.748818     0.697751     0.602929      0.0
4854        4854        2.0       200.0      0.3       100.0      0.90  0.001  35.0   0.748520     0.707640     0.623501      0.0
4066        4066        2.0       200.0      0.2       100.0      0.99  0.001  27.0   0.747545     0.696917     0.588816      0.0
5010        5010        2.0       200.0      0.3       100.0      0.99  0.001  35.0   0.747417     0.692161     0.599719      0.0
3918        3918        2.0       200.0      0.2       100.0      0.90  0.001  35.0   0.747229     0.695494     0.626002      0.0
4698        4698        2.0       200.0      0.3       100.0      0.80  0.001  35.0  

##My Data

In [ ]:
name="Spark"
tipo="ORIGIN+MESSAGE"

In [ ]:
# f=open(pathname+f"/Input/{name}/{tipo}/{name}_InputData_{tipo}.txt","r")
# lines=f.readlines()
# data=[]
# token_lists=[]
# idx_in=[]
# for i,line in enumerate(lines):
#   line=line.rstrip("\n")
#   row_tk=[]
#   for el in line.split(", "):
#     el=el.replace("[","")
#     el=el.replace("]","")
#     el=el.replace("'","")
#     el=el.replace(" ","_")
#     row_tk.append(el)
#   sent=" ".join(row_tk)
#   data.append(sent)
#   token_lists.append(row_tk)
#   idx_in.append(i)
# f.close()
# print(len(data))

NameError: ignored

In [ ]:
print("LOAD DATA")

filename = pathname+f"/Input/{name}/{tipo}/{name}_InputData_{tipo}.txt"
print(f"READ FILE: {filename}")
data=[]

with open(filename, 'r') as csvfile:
    rows = csv.reader(csvfile, delimiter='\t')
    for i,row in enumerate(rows):
        tmp = row[0].replace("[", "")
        tmp = tmp.replace("]", "")
        tmp = tmp.replace("'", "")
        row_l = list(tmp.split(", "))
        if i<30:
            print(i, row_l)
        data.append(row_l)
print("END LOAD DATA")

LOAD DATA
READ FILE: /content/gdrive/MyDrive/Colab Notebooks/ProdLDA/Input/Spark/ORIGIN+MESSAGE/Spark_InputData_ORIGIN+MESSAGE.txt
0 ['broadcast', 'torrent', 'broadcast', 'read', 'broadcast', 'variable', 'take', 'broadcast torrent', 'broadcast torrent', 'broadcast read', 'broadcast read', 'broadcast variable', 'variable take', 'torrent broadcast read', 'read broadcast variable', 'broadcast variable take']
1 ['broadcast', 'torrent', 'broadcast', 'start', 'read', 'broadcast', 'variable', 'broadcast torrent', 'broadcast torrent', 'broadcast start', 'start read', 'broadcast read', 'broadcast variable', 'torrent broadcast start', 'broadcast start read', 'start read broadcast', 'read broadcast variable']
2 ['cache', 'partition', 'not', 'find', 'compute', 'cache manager', 'manager partition', 'partition not', 'not find', 'find compute', 'cache manager partition', 'manager partition not', 'partition not find', 'not find compute']
3 ['configuration', 'deprecate', 'mapred', 'deprecate', 'use', '

In [ ]:
MAX_TOPIC=len(data)
print(MAX_TOPIC, type(data))

45 <class 'list'>


In [ ]:
f=open(pathname+f"/Input/{name}/{tipo}/{name}_Raw_{tipo}.txt","r")
lines=f.readlines()
originals=[]
for i,line in enumerate(lines):
  line=line.rstrip("\n")
  print(line)
  originals.append(line)
f.close()
print(len(originals))

broadcast.TorrentBroadcast: Reading broadcast variable 9 took 73 ms
broadcast.TorrentBroadcast: Started reading broadcast variable 9
spark.CacheManager: Partition rdd_2_6 not found, computing it
Configuration.deprecation: mapred.job.id is deprecated. Instead, use mapreduce.job.id
Configuration.deprecation: mapred.task.id is deprecated. Instead, use mapreduce.task.attempt.id
Configuration.deprecation: mapred.task.is.map is deprecated. Instead, use mapreduce.task.ismap
Configuration.deprecation: mapred.task.partition is deprecated. Instead, use mapreduce.task.partition
Configuration.deprecation: mapred.tip.id is deprecated. Instead, use mapreduce.task.id
executor.CoarseGrainedExecutorBackend: Connecting to driver: spark://CoarseGrainedScheduler@10.10.34.11:59027
executor.CoarseGrainedExecutorBackend: Driver commanded a shutdown
executor.CoarseGrainedExecutorBackend: Driver 10.10.34.11:59027 disassociated! Shutting down.
executor.CoarseGrainedExecutorBackend: Got assigned task 6
executor.

Preprocessing

In [ ]:
TotWord=[]
for el in enumerate(data): #UNIQUE DATA
    #print(el[1])
    # print(lst)
    for wrd in el[1]:
        # print(wrd)
        TotWord.append(wrd)
vocabulary=set(TotWord)
print(len(vocabulary))

787


CREATE FOLDER FOR ProdLDA

In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/Colab Notebooks/ProdLDA'

In [ ]:
os.mkdir(f"{name}_data_folder_{tipo}")

In [ ]:
f=open(f"{name}_data_folder_{tipo}/vocabulary.txt","w")
for el in vocabulary:
  tmp=[]
  splits=el.split(" ")
  conc_w="_".join(splits)
  #print(conc_w)
  print(conc_w, file=f)
f.close()

In [ ]:
f=open(f"{name}_data_folder_{tipo}/corpus.tsv","w")
to_print=[]
for tw in data:
    tmp=[]
    for el in tw:
        splits=el.split(" ")
        tmp.append("_".join(splits))
    only_string=" ".join(tmp)
    print(only_string+"\t"+"train",file=f)
    to_print.append(only_string)
f.close()

In [ ]:
print(to_print[0])
print(data[0])

acknowledge active namenode pool service acknowledge_active active_namenode namenode_block block_pool pool_service acknowledge_active_namenode active_namenode_block namenode_block_pool block_pool_service
['acknowledge', 'active', 'namenode', 'pool', 'service', 'acknowledge active', 'active namenode', 'namenode block', 'block pool', 'pool service', 'acknowledge active namenode', 'active namenode block', 'namenode block pool', 'block pool service']


# OCTIS

In [ ]:
from octis.dataset.dataset import Dataset
from octis.models.LDA import LDA
from octis.evaluation_metrics.coherence_metrics import Coherence

# Load a dataset
dataset = Dataset()
dataset.load_custom_dataset_from_folder(pathname+f"/{name}_data_folder_{tipo}")#fetch_dataset("20NewsGroup")

In [ ]:
len(dataset.get_vocabulary())

399

In [ ]:
len(dataset.get_corpus())

45

In [ ]:
from octis.models.ProdLDA import ProdLDA
from octis.models.pytorchavitm import AVITM

## INFERENCE NETWORK

In [ ]:
"""PyTorch class for feed foward inference network."""

from collections import OrderedDict
from torch import nn
import torch


class myInferenceNetwork(nn.Module):

    """Inference Network."""

    def __init__(self, input_size, output_size, hidden_sizes, activation='softplus', dropout=0.2):

        super(myInferenceNetwork, self).__init__()
        assert isinstance(input_size, int), "input_size must by type int."
        assert isinstance(output_size, int), "output_size must be type int."
        assert isinstance(hidden_sizes, tuple), \
            "hidden_sizes must be type tuple."
        assert activation in ['softplus', 'relu', 'sigmoid', 'tanh', 'leakyrelu',
                              'rrelu', 'elu', 'selu'], \
            "activation must be 'softplus', 'relu', 'sigmoid', 'leakyrelu'," \
            " 'rrelu', 'elu', 'selu' or 'tanh'."
        assert dropout >= 0, "dropout must be >= 0."

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.dropout = dropout

        if activation == 'softplus':
            self.activation = nn.Softplus()
        elif activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'leakyrelu':
            self.activation = nn.LeakyReLU()
        elif activation == 'rrelu':
            self.activation = nn.RReLU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        elif activation == 'selu':
            self.activation = nn.SELU()

        self.input_layer = nn.Linear(input_size, hidden_sizes[0])


        self.hiddens = nn.Sequential(OrderedDict([
            ('l_{}'.format(i), nn.Sequential(nn.Linear(h_in, h_out), self.activation))
            for i, (h_in, h_out) in enumerate(zip(hidden_sizes[:-1], hidden_sizes[1:]))]))

        self.f_mu = nn.Linear(hidden_sizes[-1], output_size)
        self.f_mu_batchnorm = nn.BatchNorm1d(output_size, affine=False)

        self.f_sigma = nn.Linear(hidden_sizes[-1], output_size)
        self.f_sigma_batchnorm = nn.BatchNorm1d(output_size, affine=False)

        self.dropout_enc = nn.Dropout(p=self.dropout)

    def forward(self, x):
        """Forward pass."""
        x = self.input_layer(x)
        x = self.activation(x)
        x = self.hiddens(x)
        x = self.dropout_enc(x)
        mu = self.f_mu_batchnorm(self.f_mu(x))
        log_sigma = self.f_sigma_batchnorm(self.f_sigma(x))

        return mu, log_sigma


class Swish(nn.Module):
    def __init__(self, slope = 1):
        super().__init__()
        #self.slope = slope * torch.nn.Parameter(torch.ones(1))

    def forward(self, x):
        return x * torch.sigmoid(x) #self.slope *

## DECODER NETWORK

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torchsummary import summary
#from octis.models.pytorchavitm.avitm.inference_network import InferenceNetwork


class myDecoderNetwork(nn.Module):

    """AVITM Network."""

    def __init__(self, input_size, n_components=10, model_type='prodLDA',
                 hidden_sizes=(100,100), activation='softplus', dropout=0.2,
                 learn_priors=True, topic_prior_mean=0.0, topic_prior_variance=None):

        super(myDecoderNetwork, self).__init__()
        assert isinstance(input_size, int), "input_size must by type int."
        assert isinstance(n_components, int) and n_components > 0, \
            "n_components must be type int > 0."
        assert model_type in ['prodLDA', 'LDA'], \
            "model type must be 'prodLDA' or 'LDA'"
        assert isinstance(hidden_sizes, tuple), \
            "hidden_sizes must be type tuple."
        assert activation in ['softplus', 'relu', 'sigmoid', 'tanh', 'leakyrelu',
                              'rrelu', 'elu', 'selu'], \
            "activation must be 'softplus', 'relu', 'sigmoid', 'leakyrelu'," \
            " 'rrelu', 'elu', 'selu' or 'tanh'."
        assert dropout >= 0, "dropout must be >= 0."
        assert isinstance(topic_prior_mean, float), \
            "topic_prior_mean must be type float"
        # and topic_prior_variance >= 0, \
        #assert isinstance(topic_prior_variance, float), \
        #    "topic prior_variance must be type float"

        self.input_size = input_size
        self.n_components = n_components
        self.model_type = model_type
        self.hidden_sizes = hidden_sizes
        self.activation = activation
        self.dropout = dropout
        self.learn_priors = learn_priors

        self.inf_net = myInferenceNetwork(input_size, n_components, hidden_sizes, activation)
        if torch.cuda.is_available():
            self.inf_net = self.inf_net.cuda()

        # init prior parameters
        # \mu_1k = log \alpha_k + 1/K \sum_i log \alpha_i;
        # \alpha = 1 \forall \alpha

        #self.topic_prior_mean = topic_prior_mean
        self.prior_mean = torch.tensor([topic_prior_mean] * n_components)
        if torch.cuda.is_available():
            self.prior_mean = self.prior_mean.cuda()
        if self.learn_priors:
            self.prior_mean = nn.Parameter(self.prior_mean)
        # \Sigma_1kk = 1 / \alpha_k (1 - 2/K) + 1/K^2 \sum_i 1 / \alpha_k;
        # \alpha = 1 \forall \alpha

        if topic_prior_variance is None:
            topic_prior_variance = 1. - (1. / self.n_components)
        self.prior_variance = torch.tensor([topic_prior_variance] * n_components)
        if torch.cuda.is_available():
            self.prior_variance = self.prior_variance.cuda()
        if self.learn_priors:
            self.prior_variance = nn.Parameter(self.prior_variance)

        self.beta = torch.Tensor(n_components, input_size)
        print("BETA SHAPE:",n_components, input_size)
        if torch.cuda.is_available():
            self.beta = self.beta.cuda()
        self.beta = nn.Parameter(self.beta)
        nn.init.xavier_uniform_(self.beta)

        self.beta_batchnorm = nn.BatchNorm1d(input_size, affine=False)

        # dropout on theta
        self.drop_theta = nn.Dropout(p=self.dropout)


    @staticmethod
    def reparameterize(mu, logvar):
        """Reparameterize the theta distribution."""
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def forward(self, x):
        """Forward pass."""
        # batch_size x n_components(num_topics)
        posterior_mu, posterior_log_sigma = self.inf_net(x)
        print("\nFORWARD DECODER")
        print(posterior_mu)
        print(list(posterior_mu.size())) #len_sent x n_topic
        print(posterior_log_sigma)
        print(list(posterior_log_sigma.size())) #len_sent x n_topic
        posterior_sigma = torch.exp(posterior_log_sigma)

        # generate samples from theta
        theta = F.softmax(self.reparameterize(posterior_mu, posterior_log_sigma), dim=1)
        print(theta) 
        print(list(theta.size())) #len_sent x n_topic
        topic_doc = theta
        theta = self.drop_theta(theta)

        # prodLDA vs LDA
        if self.model_type == 'prodLDA':
            # in: batch_size x input_size x n_components
            word_dist = F.softmax(self.beta_batchnorm(torch.matmul(theta, self.beta)), dim=1) #wn|beta,theta
            print(word_dist)
            print(list(word_dist.size())) #len_sent x tot_words
            topic_word = self.beta
            print(self.beta)
            print(list(self.beta.size())) #n_topic x tot_words
            # word_dist: batch_size x input_size
            self.topic_word_matrix = self.beta


        return self.prior_mean, self.prior_variance, posterior_mu, posterior_sigma, posterior_log_sigma, word_dist, topic_word,topic_doc

    def get_theta(self, x):
        with torch.no_grad():
            # batch_size x n_components
            posterior_mu, posterior_log_sigma = self.inf_net(x)
            posterior_sigma = torch.exp(posterior_log_sigma)

            # generate samples from theta
            theta = F.softmax(self.reparameterize(posterior_mu, posterior_log_sigma), dim=1)

            return theta

## AVITM_MODEL

In [ ]:
"""Class to train AVITM models."""

import datetime
import os
from collections import defaultdict

import numpy as np
import torch
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from octis.models.early_stopping.pytorchtools import EarlyStopping
from octis.models.pytorchavitm.avitm.decoder_network import DecoderNetwork

from tqdm import tqdm

class myAVITM_model(object):

    def __init__(self, input_size, num_topics=10, model_type='prodLDA', hidden_sizes=(100, 100),
                 activation='softplus', dropout=0.2, learn_priors=True, batch_size=64, lr=2e-3, momentum=0.99,
                 solver='adam', num_epochs=100, reduce_on_plateau=False, topic_prior_mean=0.0,
                 topic_prior_variance=None, num_samples=10, num_data_loader_workers=0, verbose=False):


        assert isinstance(input_size, int) and input_size > 0, \
            "input_size must by type int > 0."
        assert isinstance(num_topics, int) and input_size > 0, \
            "num_topics must by type int > 0."
        assert model_type in ['LDA', 'prodLDA'], \
            "model must be 'LDA' or 'prodLDA'."
        assert isinstance(hidden_sizes, tuple), \
            "hidden_sizes must be type tuple."
        assert activation in ['softplus', 'relu', 'sigmoid', 'swish', 'tanh', 'leakyrelu',
                              'rrelu', 'elu', 'selu'], \
            "activation must be 'softplus', 'relu', 'sigmoid', 'swish', 'leakyrelu'," \
            " 'rrelu', 'elu', 'selu' or 'tanh'."
        assert dropout >= 0, "dropout must be >= 0."
        # assert isinstance(learn_priors, bool), "learn_priors must be boolean."
        assert isinstance(batch_size, int) and batch_size > 0, \
            "batch_size must be int > 0."
        assert lr > 0, "lr must be > 0."
        assert isinstance(momentum, float) and 0 < momentum <= 1, \
            "momentum must be 0 < float <= 1."
        assert solver in ['adagrad', 'adam', 'sgd', 'adadelta', 'rmsprop'], \
            "solver must be 'adam', 'adadelta', 'sgd', 'rmsprop' or 'adagrad'"
        assert isinstance(reduce_on_plateau, bool), \
            "reduce_on_plateau must be type bool."
        assert isinstance(topic_prior_mean, float), \
            "topic_prior_mean must be type float"
        # and topic_prior_variance >= 0, \
        # assert isinstance(topic_prior_variance, float), \
        #    "topic prior_variance must be type float"

        self.input_size = input_size
        self.num_topics = num_topics
        self.verbose = verbose
        self.model_type = model_type
        self.hidden_sizes = hidden_sizes
        self.activation = activation
        self.dropout = dropout
        self.learn_priors = learn_priors
        self.batch_size = batch_size
        self.lr = lr
        self.momentum = momentum
        self.solver = solver
        self.num_epochs = num_epochs
        self.reduce_on_plateau = reduce_on_plateau
        self.num_data_loader_workers = num_data_loader_workers
        self.topic_prior_mean = topic_prior_mean
        self.topic_prior_variance = topic_prior_variance
        self.num_samples = num_samples
        # init inference avitm network
        self.model = myDecoderNetwork(
            input_size, num_topics, model_type, hidden_sizes, activation,
            dropout, learn_priors, topic_prior_mean, topic_prior_variance)
        self.early_stopping = EarlyStopping(patience=5, verbose=True)
        self.validation_data = None
        # init optimizer
        if self.solver == 'adam':
            self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(self.momentum, 0.99))
        elif self.solver == 'sgd':
            self.optimizer = optim.SGD(self.model.parameters(), lr=lr, momentum=self.momentum)
        elif self.solver == 'adagrad':
            self.optimizer = optim.Adagrad(self.model.parameters(), lr=lr)
        elif self.solver == 'adadelta':
            self.optimizer = optim.Adadelta(self.model.parameters(), lr=lr)
        elif self.solver == 'rmsprop':
            self.optimizer = optim.RMSprop(self.model.parameters(), lr=lr, momentum=self.momentum)
        # init lr scheduler
        if self.reduce_on_plateau:
            self.scheduler = ReduceLROnPlateau(self.optimizer, patience=5, verbose=True)

        # performance attributes
        self.best_loss_train = float('inf')

        # training atributes
        self.model_dir = None
        self.train_data = None
        self.nn_epoch = None

        # learned topics
        self.best_components = None

        # Use cuda if available
        if torch.cuda.is_available():
            self.USE_CUDA = True
        else:
            self.USE_CUDA = False
        if self.USE_CUDA:
            self.model = self.model.cuda()

    def _loss(self, inputs, word_dists, prior_mean, prior_variance,
              posterior_mean, posterior_variance, posterior_log_variance):
        # KL term
        # var division term
        var_division = torch.sum(posterior_variance / prior_variance, dim=1)
        # diff means term
        diff_means = prior_mean - posterior_mean
        diff_term = torch.sum( (diff_means * diff_means) / prior_variance, dim=1)
        # logvar det division term
        logvar_det_division = \
            prior_variance.log().sum() - posterior_log_variance.sum(dim=1)
        # combine terms
        KL = 0.5 * (var_division + diff_term - self.num_topics + logvar_det_division)
        # Reconstruction term
        RL = -torch.sum(inputs * torch.log(word_dists + 1e-10), dim=1)
        loss = KL + RL

        return loss.sum()

    def _train_epoch(self, loader):
        """Train epoch."""
        self.model.train()
        train_loss = 0
        samples_processed = 0
        topic_doc_list = []
        for batch_samples in loader:
            # batch_size x vocab_size
            x = batch_samples['X']

            if self.USE_CUDA:
                x = x.cuda()
            # forward pass
            self.model.zero_grad()
            prior_mean, prior_var, posterior_mean, posterior_var, posterior_log_var, word_dists, topic_words, topic_document = self.model(x)

            topic_doc_list.extend(topic_document)

            # backward pass
            loss = self._loss(x, word_dists, prior_mean, prior_var, posterior_mean, posterior_var, posterior_log_var)
            loss.backward()
            self.optimizer.step()

            # compute train loss
            samples_processed += x.size()[0]
            train_loss += loss.item()

        train_loss /= samples_processed

        return samples_processed, train_loss, topic_words, topic_doc_list

    def _validation(self, loader):
        """Train epoch."""
        self.model.eval()
        val_loss = 0
        samples_processed = 0
        for batch_samples in loader:
            # batch_size x vocab_size
            x = batch_samples['X']

            if self.USE_CUDA:
                x = x.cuda()
            # forward pass
            self.model.zero_grad()
            prior_mean, prior_var, posterior_mean, posterior_var, posterior_log_var, word_dists, topic_word, topic_document = self.model(x)

            loss = self._loss(x, word_dists, prior_mean, prior_var,  posterior_mean, posterior_var, posterior_log_var)

            # compute train loss
            samples_processed += x.size()[0]
            val_loss += loss.item()

        val_loss /= samples_processed

        return samples_processed, val_loss

    def fit(self, train_dataset, validation_dataset, save_dir=None):


        self.model_dir = save_dir
        self.train_data = train_dataset
        self.validation_data = validation_dataset
        train_loader = DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True,num_workers=self.num_data_loader_workers)

        # init training variables
        train_loss = 0
        samples_processed = 0

        #bar = trange(self.num_epochs)

        # train loop
        for epoch in range(self.num_epochs):#bar:
            self.nn_epoch = epoch
            # train epoch
            s = datetime.datetime.now()
            sp, train_loss, topic_words, topic_document = self._train_epoch(train_loader)
            samples_processed += sp
            e = datetime.datetime.now()

            # report
            #print("Epoch: [{}/{}]\tSamples: [{}/{}]\tTrain Loss: {}\tTime: {}".format(
            #    epoch + 1, self.num_epochs, samples_processed,
            #    len(self.train_data) * self.num_epochs, train_loss, e - s))

            self.best_components = self.model.beta
            self.final_topic_word = topic_words
            self.final_topic_document = topic_document
            self.best_loss_train = train_loss
            #bar.set_description("Epoch: [{}/{}]\tSamples: [{}/{}]\tTrain Loss: {}\tTime: {}".format(
            #     epoch+ 1, self.num_epochs, samples_processed,
            #    len(self.train_data) * self.num_epochs, train_loss, e - s))
            if self.validation_data is not None:
                validation_loader = DataLoader(
                    self.validation_data, batch_size=self.batch_size, shuffle=True,
                    num_workers=self.num_data_loader_workers)
                # train epoch
                s = datetime.datetime.now()
                val_samples_processed, val_loss = self._validation(validation_loader)
                e = datetime.datetime.now()

                # report
                #print("Epoch: [{}/{}]\tSamples: [{}/{}]\tValidation Loss: {}\tTime: {}".format(
                #    epoch + 1, self.num_epochs, val_samples_processed,
                #    len(self.validation_data) * self.num_epochs, val_loss, e - s))

                if np.isnan(val_loss) or np.isnan(train_loss):
                    break
                else:
                    self.early_stopping(val_loss, self.model)
                    if self.early_stopping.early_stop:
                        print("Early stopping")
                        if save_dir is not None:
                            self.save(save_dir)
                        break
            

    def predict(self, dataset):
        """Predict input."""
        self.model.eval()

        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False,
                            num_workers=self.num_data_loader_workers)

        topic_document_mat = []
        with torch.no_grad():
            for batch_samples in loader:
                # batch_size x vocab_size
                x = batch_samples['X']
                x = x.reshape(x.shape[0], -1)
                if self.USE_CUDA:
                    x = x.cuda()
                # forward pass
                self.model.zero_grad()
                _, _, _, _, _, _, _, topic_document = self.model(x)
                topic_document_mat.append(topic_document)

        results = self.get_info()
        # results['test-topic-document-matrix2'] = np.vstack(
        #    np.asarray([i.cpu().detach().numpy() for i in topic_document_mat])).T
        results['test-topic-document-matrix'] = np.asarray(self.get_thetas(dataset)).T

        return results

    def get_topic_word_mat(self):
        top_wor = self.final_topic_word.cpu().detach().numpy()
        return top_wor

    def get_topic_document_mat(self):
        top_doc = self.final_topic_document
        top_doc_arr = np.array([i.cpu().detach().numpy() for i in top_doc])
        return top_doc_arr

    def get_topics(self, k=20):
        """
        Retrieve topic words.

        Args
            k : (int) number of words to return per topic, default 10.
        """
        assert k <= self.input_size, "k must be <= input size."
        component_dists = self.best_components
        topics = defaultdict(list)
        topics_list = []
        if self.num_topics is not None:
            for i in range(self.num_topics):
                _, idxs = torch.topk(component_dists[i], k)
                component_words = [self.train_data.idx2token[idx]
                                   for idx in idxs.cpu().numpy()]
                topics[i] = component_words
                topics_list.append(component_words)

        return topics_list

    def get_info(self):
        info = {}
        topic_word = self.get_topics()
        topic_word_dist = self.get_topic_word_mat()
        topic_document_dist = self.get_topic_document_mat()
        info['topics'] = topic_word

        info['topic-document-matrix2'] = topic_document_dist.T
        info['topic-document-matrix'] = np.asarray(self.get_thetas(self.train_data)).T

        info['topic-word-matrix'] = topic_word_dist
        return info

    def _format_file(self):
        model_dir = "AVITM_nc_{}_tpm_{}_tpv_{}_hs_{}_ac_{}_do_{}_lr_{}_mo_{}_rp_{}". \
            format(self.num_topics, 0.0, 1 - (1. / self.num_topics),
                   self.model_type, self.hidden_sizes, self.activation,
                   self.dropout, self.lr, self.momentum,
                   self.reduce_on_plateau)
        return model_dir

    def save(self, models_dir=None):

        if (self.model is not None) and (models_dir is not None):

            model_dir = self._format_file()
            if not os.path.isdir(os.path.join(models_dir, model_dir)):
                os.makedirs(os.path.join(models_dir, model_dir))

            filename = "epoch_{}".format(self.nn_epoch) + '.pth'
            fileloc = os.path.join(models_dir, model_dir, filename)
            with open(fileloc, 'wb') as file:
                torch.save({'state_dict': self.model.state_dict(),
                            'dcue_dict': self.__dict__}, file)

    def load(self, model_dir, epoch):

        epoch_file = "epoch_" + str(epoch) + ".pth"
        model_file = os.path.join(model_dir, epoch_file)
        with open(model_file, 'rb') as model_dict:
            checkpoint = torch.load(model_dict)

        for (k, v) in checkpoint['dcue_dict'].items():
            setattr(self, k, v)

        self.model.load_state_dict(checkpoint['state_dict'])

    def get_thetas(self, dataset):
        """
        Get the document-topic distribution for a dataset of topics. Includes multiple sampling to reduce variation via
        the parameter num_samples.
        :param dataset: a PyTorch Dataset containing the documents
        """
        self.model.eval()

        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_data_loader_workers)
        final_thetas = []
        for sample_index in range(self.num_samples):
            with torch.no_grad():
                collect_theta = []
                for batch_samples in loader:
                    # batch_size x vocab_size
                    x = batch_samples['X']
                    x = x.reshape(x.shape[0], -1)
                    if self.USE_CUDA:
                        x = x.cuda()
                    # forward pass
                    self.model.zero_grad()
                    collect_theta.extend(self.model.get_theta(x).cpu().numpy().tolist())

                final_thetas.append(np.array(collect_theta))
        return np.sum(final_thetas, axis=0) / self.num_samples

## AVITM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from octis.models.model import AbstractModel
from octis.models.pytorchavitm import datasets
#from octis.models.pytorchavitm.avitm import avitm_model


class myAVITM(AbstractModel):

    def __init__(self, num_topics=10, model_type='prodLDA', activation='softplus',
                 dropout=0.2, learn_priors=True, batch_size=64, lr=2e-3, momentum=0.99,
                 solver='adam', num_epochs=100, reduce_on_plateau=False, prior_mean=0.0,
                 prior_variance=None, num_layers=2, num_neurons=100, num_samples=10,
                 use_partitions=False):

        super().__init__()
        self.hyperparameters['num_topics'] = num_topics
        self.hyperparameters['model_type'] = model_type
        self.hyperparameters['activation'] = activation
        self.hyperparameters['dropout'] = dropout
        self.hyperparameters['learn_priors'] = learn_priors
        self.hyperparameters['batch_size'] = batch_size
        self.hyperparameters['lr'] = lr
        self.hyperparameters['momentum'] = momentum
        self.hyperparameters['solver'] = solver
        self.hyperparameters['num_epochs'] = num_epochs
        self.hyperparameters['reduce_on_plateau'] = reduce_on_plateau
        self.hyperparameters["prior_mean"] = prior_mean
        self.hyperparameters["prior_variance"] = prior_variance
        self.hyperparameters["num_neurons"] = num_neurons
        self.hyperparameters["num_layers"] = num_layers
        self.hyperparameters["num_samples"] = num_samples

        hidden_sizes = tuple([num_neurons for _ in range(num_layers)])
        self.use_partitions = use_partitions
        self.hyperparameters['hidden_sizes'] = tuple(hidden_sizes)
        self.X_train=None

    def get_hyperparameters(self):
        return self.hyperparameters

    def train_model(self, dataset, hyperparameters=None, top_words=10):

        if hyperparameters is None:
            hyperparameters = {}
        self.set_params(hyperparameters)


        self.vocab = dataset.get_vocabulary()
        data_corpus = [' '.join(i) for i in dataset.get_corpus()]
        x_train, input_size, docbow = self.preprocess(self.vocab, train=data_corpus)

        self.model = myAVITM_model(
            input_size=input_size, num_topics=self.hyperparameters['num_topics'],
            model_type=self.hyperparameters['model_type'], hidden_sizes=self.hyperparameters['hidden_sizes'],
            activation=self.hyperparameters['activation'], dropout=self.hyperparameters['dropout'],
            learn_priors=self.hyperparameters['learn_priors'], batch_size=self.hyperparameters['batch_size'],
            lr=self.hyperparameters['lr'], momentum=self.hyperparameters['momentum'],
            solver=self.hyperparameters['solver'], num_epochs=self.hyperparameters['num_epochs'],
            reduce_on_plateau=self.hyperparameters['reduce_on_plateau'], num_samples=self.hyperparameters[
                'num_samples'], topic_prior_mean=self.hyperparameters["prior_mean"],
            topic_prior_variance=self.hyperparameters["prior_variance"]
        )
        

        self.model.fit(x_train, None)
        result = self.model.get_info()

        return result

    def set_params(self, hyperparameters):
        self.hyperparameters['num_topics'] = \
            int(hyperparameters.get('num_topics', self.hyperparameters['num_topics']))
        self.hyperparameters['model_type'] = \
            hyperparameters.get('model_type', self.hyperparameters['model_type'])
        self.hyperparameters['activation'] = \
            hyperparameters.get('activation', self.hyperparameters['activation'])
        self.hyperparameters['dropout'] = float(hyperparameters.get('dropout', self.hyperparameters['dropout']))
        self.hyperparameters['learn_priors'] = \
            hyperparameters.get('learn_priors', self.hyperparameters['learn_priors'])
        self.hyperparameters['batch_size'] = \
            int(hyperparameters.get('batch_size', self.hyperparameters['batch_size']))
        self.hyperparameters['lr'] = float(hyperparameters.get('lr', self.hyperparameters['lr']))
        self.hyperparameters['momentum'] = \
            float(hyperparameters.get('momentum', self.hyperparameters['momentum']))
        self.hyperparameters['solver'] = hyperparameters.get('solver', self.hyperparameters['solver'])
        self.hyperparameters['num_epochs'] = \
            int(hyperparameters.get('num_epochs', self.hyperparameters['num_epochs']))
        self.hyperparameters['reduce_on_plateau'] = \
            hyperparameters.get('reduce_on_plateau', self.hyperparameters['reduce_on_plateau'])
        self.hyperparameters["prior_mean"] = \
            hyperparameters.get('prior_mean', self.hyperparameters['prior_mean'])
        self.hyperparameters["prior_variance"] = \
            hyperparameters.get('prior_variance', self.hyperparameters['prior_variance'])

        self.hyperparameters["num_layers"] = \
            int(hyperparameters.get('num_layers', self.hyperparameters['num_layers']))
        self.hyperparameters["num_neurons"] = \
            int(hyperparameters.get('num_neurons', self.hyperparameters['num_neurons']))

        self.hyperparameters['hidden_sizes'] = tuple(
            [self.hyperparameters["num_neurons"] for _ in range(self.hyperparameters["num_layers"])])

    def inference(self, x_test):
        assert isinstance(self.use_partitions, bool) and self.use_partitions
        results = self.model.predict(x_test)
        return results

    def partitioning(self, use_partitions=False):
        self.use_partitions = use_partitions
      
    def get_X_train(self):
        return self.X_train

    @staticmethod
    def preprocess(vocab, train, test=None, validation=None):
        vocab2id = {w: i for i, w in enumerate(vocab)}
        vec = CountVectorizer()#vocabulary=vocab2id, token_pattern=r'(?u)\b\w+\b')
        entire_dataset = train.copy()

        if test is not None:
            entire_dataset.extend(test)
        if validation is not None:
            entire_dataset.extend(validation)

        vec.fit(entire_dataset)
        idx2token = {v: k for (k, v) in vec.vocabulary_.items()}
        #for el in train:
        #  print(el)
        X_train = vec.transform(train) #n_sentences x n_tot_words
        print(X_train.shape)
        train_data = datasets.BOWDataset(X_train.toarray(), idx2token) #TRASFORMA IN TORCH TENSOR
        #print(train_data)
        input_size = len(idx2token.keys())
        print(input_size) #LEN TOT WORDS


        if test is None and validation is None:
            print("BOTH NONE")
            return train_data, input_size,X_train


## PROD_LDA

In [ ]:
#from octis.models.pytorchavitm.AVITM import AVITM


class myProdLDA(myAVITM):
    def __init__(self, num_topics=10, activation='softplus', dropout=0.2, learn_priors=True, batch_size=64, lr=2e-3,
                 momentum=0.99, solver='adam', num_epochs=100, reduce_on_plateau=False, prior_mean=0.0,
                 prior_variance=None, num_layers=2, num_neurons=100, num_samples=10, use_partitions=True):
        super().__init__(num_topics=num_topics, model_type='prodLDA', activation=activation, dropout=dropout,
                         learn_priors=learn_priors, batch_size=batch_size, lr=lr, momentum=momentum,
                         solver=solver, num_epochs=num_epochs, reduce_on_plateau=reduce_on_plateau,
                         prior_mean=prior_mean, prior_variance=prior_variance, num_layers=num_layers,
                         num_neurons=num_neurons, num_samples=num_samples, use_partitions=use_partitions)

    def train_model(self, dataset, hyperparameters=None, top_words=10):
        return super().train_model(dataset, hyperparameters, top_words)

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
avitm = myProdLDA(num_topics=30, batch_size=128)
avitm.partitioning()
trained_model=avitm.train_model(dataset)

Output streaming troncato alle ultime 5000 righe.
         7.2719e-04],
        [3.1606e-03, 2.3052e-03, 1.5975e-03,  ..., 1.5125e-03, 1.7616e-03,
         1.5578e-03]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
[45, 399]
Parameter containing:
tensor([[-0.0275,  0.0092, -0.1261,  ...,  0.0361, -0.0356, -0.0315],
        [ 0.0656, -0.0207,  0.1010,  ..., -0.0008,  0.0364,  0.0141],
        [-0.0972, -0.0528, -0.0089,  ...,  0.0566, -0.0274,  0.0504],
        ...,
        [ 0.0574, -0.0786, -0.0929,  ..., -0.0393,  0.0735, -0.1142],
        [-0.0817,  0.0839,  0.0381,  ..., -0.0548, -0.0485, -0.0281],
        [-0.0144,  0.0787, -0.0240,  ..., -0.1208, -0.0418, -0.0363]],
       device='cuda:0', requires_grad=True)
[30, 399]

FORWARD DECODER
tensor([[ 0.1184, -2.1351,  0.0774,  ...,  0.0318, -0.0881,  1.1040],
        [ 1.2688,  1.6520, -0.7172,  ...,  0.1665, -0.5561,  1.7282],
        [ 1.6524,  1.6182, -0.5976,  ..., -0.1317,  2.5183, -0.5375],
        ...,
        [-0.5185,  0.0981

## TUNING

In [ ]:
import pickle
f=open(pathname+f"/Input/{name}/{tipo}/{name}_w2vdict_{tipo}.pkl","rb")
modelW2V=pickle.load(f)#Word2Vec.load(pathname+"WORD2VEC/word2vec_SPLIT.model")
#print(modelW2V)
print(type(modelW2V))
print(modelW2V.keys())
print(len(modelW2V.keys()))

<class 'dict'>
dict_keys(['service_try_claim', 'src_dest', 'not_replicate_block', 'eof_exception_detail', 'exception_offer_service', 'time_scan', 'add_new', 'service_register_nn', 'record_length', 'block_receiver', 'signer_secret', 'volume_storage', 'dfs', 'ipc', 'scan', 'ipc_server_responder', 'start_interval', 'persist', 'blockreport_interval', 'open_stream', 'add_filter', 'exception_java', 'slow_manage', 'storage', 'analyze_storage_directory', 'local', 'shorter_namenode_record', 'trash', 'timeout_wait_channel', 'command_action_register', 'block_pool_find', 'pool_service_begin', 'total_block_miss', 'responder_start', 'thread_balance', 'max_lock_memory', 'random_secret', 'blockreport_interval_cachereport', 'initialize', 'cache', 'write_packet_mirror', 'action_register_active', 'block_volume', 'command', 'pool_service_try', 'start_offer', 'scan_block', 'manage_writer_os', 'read_java_channel', 'new_datanode', 'bandwith', 'file_signer', 'queue_class', 'lock_acquire_nodename', 'operation'

In [ ]:
from scipy import spatial
def CLUSTERSIMILARITY2(tp, modelW2V, idx):
    similarities2=[]
    max=[]
    for i,top_wrd in enumerate(tp):
        tmp=tp
        tmp_sim=[]
        curr_v=np.zeros(300)
        num_words=0
        for el in top_wrd:
            word_vec=modelW2V[el]
            curr_v+=word_vec
            num_words+=1
        curr_v=curr_v/num_words
        for j,other_topic in enumerate(tmp):
            act_v=np.zeros(300)
            num_words=0
            for el in other_topic:
                word_vec=modelW2V[el]
                act_v+=word_vec
                num_words+=1
            act_v=act_v/num_words
            similarity=spatial.distance.cosine(curr_v, act_v)
            similarity=round(1-similarity,4)
            flag=0
            for el in top_wrd:
                if el in other_topic:
                    flag+=1
            if flag==len(top_wrd) and i!=j and "supergroup" in top_wrd:
                print(f"ALL ELEMENT OF TOPIC {i} ARE INSIDE TOPIC {j}")
                print(i,top_wrd)
                print(j,other_topic)
                tmp_sim.append(-1)
            else:
                tmp_sim.append(similarity)
            if j!=i:
                max.append(similarity)
        similarities2.append(tmp_sim)

    max=np.array(max)
    max=-np.sort(-max)
    max5=max[:5]
    similarities2=pd.DataFrame(similarities2)
    upper_tri = similarities2.where(np.triu(np.ones(similarities2.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)] #MORE THAN 90% of WORD IN COMMON
    similarities2_dropped=similarities2.drop(columns=to_drop)
    delta_k=len(to_drop)
    if idx<0:
      print(f"Features to be dropped:{to_drop} - {len(to_drop)} - max: {max5}")
    else:
      print(f"{idx}) Features to be dropped:{to_drop} - {len(to_drop)} - max: {max5}")
    
    return delta_k,to_drop

In [ ]:
topics_range=np.arange(20,MAX_TOPIC,2)
print(topics_range)
print(len(topics_range))

[20 22 24 26 28 30 32 34 36 38 40 42 44]
13


In [ ]:
best=0
best_mean=0
best_cohe=0
best_k=0
best_ne=0
best_lr=0
best_numneu=0
df_log=pd.DataFrame(columns=["num_layer","num_neuron","dropout","num_epochs","momentum","lr","k","coherence","coherence15","coherence10","delta_k"])
idx=0
for nl in [2]:
  print("Nl"*67)
  for num_neu in [100,200]:
    print("Neu"*45)
    for dp in [0.1,0.2,0.3]:
      print("Dp"*67)
      for nume in [100,200]:
        print("Nep"*45)
        for mm in [0.8,  0.9,  0.99]:
          print("MM"*67)
          for lr in [1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1]:
            print("Lr"*67)
            for k in topics_range:
              random.seed(0)
              np.random.seed(0)
              torch.manual_seed(0)
              avitm = myProdLDA(activation="sigmoid",solver="adam",num_topics=k, batch_size=128, num_epochs=nume, lr=lr,momentum=mm, dropout=dp, num_layers=nl,num_neurons=num_neu, reduce_on_plateau=True)
              avitm.partitioning()
              trained_model=avitm.train_model(dataset)

              metric10 = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=10) # Initialize metric
              coherence_cv10 = metric10.score(trained_model)
              metric15 = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=15) # Initialize metric
              coherence_cv15 = metric15.score(trained_model)
              metric = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=20) # Initialize metric
              coherence_cv = metric.score(trained_model)
              cvs=[coherence_cv,coherence_cv15, coherence_cv10]

              # CV20: 0.73 -> CV10: 0.5
              if coherence_cv>best_cohe and np.mean(cvs)>best_mean and coherence_cv>=0.7 and coherence_cv10>0.8:
                print(f"--> [BEST] K:{k}, NL:{nl}, NUM_NEU:{num_neu}, Dp:{dp},Num_ep:{nume}, MM:{mm}, Lr:{lr} - CV20:{coherence_cv}, CV15: {coherence_cv15}, CV10:{coherence_cv10}")
                best_cohe=coherence_cv
                best_dp=dp
                best_nl=nl
                best_k=k
                best_mm=mm
                best_ne=nume
                best_lr=lr
                best_numneu=num_neu
              else:
                print(f"K:{k}, NL:{nl}, NUM_NEU:{num_neu}, Dp:{dp},Num_ep:{nume}, MM:{mm}, Lr:{lr} - CV20:{coherence_cv}, CV15: {coherence_cv15}, CV10:{coherence_cv10}")

              topics_words=trained_model['topics']
              delta_k,to_drop=CLUSTERSIMILARITY2(topics_words, modelW2V,-1)

              df_log.loc[idx]=[nl, num_neu, dp, nume, mm, lr, k, coherence_cv,coherence_cv15,coherence_cv10, delta_k]
              idx+=1

df_log.to_csv(pathname+f"/{name}_ProdLDA_Train_History_{tipo}_nl={nl}.csv")
print(f"\nBEST COMBINATION - Num Layers:{best_nl}, Num Neurons:{best_numneu}, Dropout:{best_dp}, Momentum:{best_mm}, Num Topic:{best_k}, Num Epoch: {best_ne}, LR: {best_lr} Coherence: {best_cohe}")

In [ ]:
print(f"\nBEST COMBINATION - Num Layers:{best_nl},Momentum:{best_mm}, Num Topic:{best_k}, Num Epoch: {best_ne}, LR: {best_lr} Coherence: {best_cohe}")

In [ ]:
'''
SPARK
- ORIGIN+MESSAGE: BEST COMBINATION - Num Layers:1, Num Neurons:100, Dropout:0.2, Momentum:0.99, Num Topic:18, Num Epoch: 200, LR: 0.1 Coherence: 0.7705230477452703, delta_k=3
                                     Num Layers:2, Num Neurons:100, Dropout:0.3, Momentum:0.8, Num Topic:31, Num Epoch: 200, LR: 0.005 Coherence: 0.7567644351733144
- MESSAGE:
- SPLIT: BEST COMBINATION - Num Layers:2, Num Neurons:100, Dropout:0.1, Momentum:0.8, Num Topic:33, Num Epoch: 100, LR: 0.001 Coherence: 0.739176     0.691738     0.613463
HDFS
- ORIGIN+MESSAGE:
- MESSAGE:
- SPLIT:
'''

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

In [ ]:
df_log=pd.read_csv(f"/content/gdrive/MyDrive/Colab Notebooks/ProdLDA/Spark_ProdLDA_Train_History_{tipo}_nl=2.csv")

In [ ]:
print(len(df_log))

In [ ]:
final_df0=df_log
final_df0=final_df0.sort_values(by=['coherence'], ascending=False)
print(final_df0.head(50))
print(len(final_df0))

In [ ]:
final_df0=df_log[df_log['delta_k']==0]
final_df0=final_df0.sort_values(by=['coherence'], ascending=False)
print(final_df0.head(50))
print(len(final_df0))

In [ ]:
df_log1=pd.read_csv(pathname+f"/{name}_ProdLDA_Train_History_{tipo}_nl=1.csv")
df_log2=pd.read_csv(pathname+f"/{name}_ProdLDA_Train_History_{tipo}_nl=2.csv")
df_log=pd.concat([df_log1, df_log2])
final_df0=df_log[df_log['delta_k']==0]
final_df0=final_df0.sort_values(by=['coherence'], ascending=False)
print(len(final_df0))
print(final_df0.head(30))

4058
      Unnamed: 0  num_layer  num_neuron  dropout  num_epochs  momentum     lr     k  coherence  coherence15  coherence10  delta_k
8106        8106        1.0       200.0      0.3       100.0      0.99  0.005  15.0   0.760239     0.801342     0.893225      0.0
5127        5127        1.0       200.0      0.1       100.0      0.99  0.010  21.0   0.739486     0.809316     0.843778      0.0
5076        5076        2.0       200.0      0.2       100.0      0.80  0.005  23.0   0.725051     0.772995     0.797634      0.0
6304        6304        2.0       200.0      0.3       100.0      0.80  0.001  29.0   0.707780     0.650112     0.576392      0.0
6514        6514        2.0       200.0      0.3       100.0      0.90  0.001  29.0   0.707173     0.652257     0.564941      0.0
2537        2537        2.0       100.0      0.3       100.0      0.80  0.001  55.0   0.706853     0.648550     0.559699      0.0
1277        1277        2.0       100.0      0.2       100.0      0.80  0.001  55.0  

In [ ]:
best_nl=2
best_nn=100
best_do=0.1
best_ne=100
best_mm=0.9
best_lr=0.01
best_k=22

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
print(best_nl,best_nn, best_do, best_ne, best_mm, best_lr, best_k)
avitm = myProdLDA(activation="sigmoid",solver="adam",num_topics=best_k, batch_size=128, num_epochs=best_ne, lr=best_lr,momentum=best_mm, dropout=best_do,num_layers=best_nl,\
                  num_neurons=best_nn, reduce_on_plateau=True)
avitm.partitioning()
trained_model=avitm.train_model(dataset)

metric10 = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=10) # Initialize metric
coherence_cv10 = metric10.score(trained_model)
metric15 = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=15) # Initialize metric
coherence_cv15 = metric15.score(trained_model)
metric = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=20) # Initialize metric
coherence_cv = metric.score(trained_model)
cvs=[coherence_cv,coherence_cv15, coherence_cv10]
print(coherence_cv, coherence_cv15, coherence_cv10)

topics_words=trained_model['topics']
for el in topics_words:
  print(el)
delta_k,to_drop=CLUSTERSIMILARITY2(topics_words, modelW2V,-1)

2 100 0.1 100 0.9 0.01 22
0.7255035745913038 0.7854322880420557 0.8433122502004504
['register_unix_signal', 'unix', 'signal', 'register_unix', 'handler', 'type_disk', 'unix_signal_handler', 'signal_handler', 'bpid_volume', 'volume_storage_type', 'scan_bpid', 'type', 'unix_signal', 'volume_storage', 'file_not', 'storage_type', 'not_find', 'file_not_find', 'add_volume', 'scan_bpid_volume']
['data', 'mesos', 'data_error_process', 'error_process', 'data_error', 'mesos_slave', 'error_process_unknown', 'mesos_slave_data', 'process_unknown', 'slave_data', 'operation_src_dest', 'operation_src', 'unknown_operation_src', 'unknown_operation', 'process_unknown_operation', 'unknown', 'slave', 'error', 'slave_data_error', 'operation']
['responder', 'packet_responder_terminate', 'terminate', 'packet', 'volume_scanner_exit', 'open_ipc_server', 'responder_terminate', 'transfer_transmit', 'transmit', 'receiver_write', 'data_transfer_transmit', 'packet_responder', 'slow_block_receiver', 'schedule', 'set_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
ver=1
if ver==1:
  topdoc=trained_model['topic-document-matrix']
else:
  topdoc=trained_model['topic-document-matrix2']

doctop=topdoc.T
print(topdoc.shape)
print(doctop.shape)

f=open(pathname+f"/{name}_RESULTS_PRODLDA_{tipo}.csv","w")

for i in range(len(doctop)):
  t=np.argmax(doctop[i])
  tw=topics_words[t]
  s=data[i]
  o=originals[i]
  o=o.replace(","," ")
  tw=" ".join(tw)
  s=" ".join(s)
  #print(type(t),type(tw),type(s),type(o))
  out=str(t)+","+tw+","+s+","+o
  print(str(t)+","+tw+","+o)
  print(out, file=f)
f.close()

(22, 98)
(98, 22)
15,pool_service service block_pool_service begin handshake_nn service_begin_handshake begin_handshake_nn begin_handshake pool_service_begin namenode_block active handshake register acknowledge_active_namenode state service_begin active_namenode_block acknowledge namenode_block_pool acknowledge_active,Acknowledging ACTIVE Namenode Block pool BP-108841162-10.10.34.11-1440074360971 (Datanode Uuid 78bbd6f1-d006-423d-9385-6de0f2676b45) service to mesos-master-1/10.10.34.11:9000
5,filter user context static_user_filter user_filter add_filter filter_context filter_static_user add_filter_static user_filter_context add static_user static filter_static filter_context_datanode context_datanode log filter_context_static filter_context_log context_static,Added filter static_user_filter (class=org.apache.hadoop.http.lib.StaticUserWebFilter$StaticUserFilter) to context datanode
5,filter user context static_user_filter user_filter add_filter filter_context filter_static_user add_filt

In [ ]:
#print(*list(trained_model.keys()), sep="\n") # Print the output identifiers

In [ ]:
#for t in trained_model['topics']:
#  print(len(t), " ".join(t))

In [ ]:
#metric = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=20) # Initialize metric
#coherence_cv = metric.score(trained_model) # Compute score of the metric
#print(f"Coherence:{coherence_cv}")

In [ ]:
#Spark
#Coherence:0.7176199121862619
#Coherence:0.6843914959556437

In [ ]:
range_topics

In [ ]:
best_nt=0
best_cohe=0
df_log=pd.DataFrame(columns=["k","coherence"])
idx=0
lr=3e-3
ne=500
for k in range_topics:
  torch.manual_seed(0)
  avitm = myProdLDA(activation=best_act, solver=best_solv, num_topics=k,batch_size=64, num_epochs=ne, lr=lr)#, reduce_on_plateau=True)
  avitm.partitioning()
  trained_model=avitm.train_model(dataset)
  metric = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=20) # Initialize metric
  coherence_cv = metric.score(trained_model) # Compute score of the metric
  df_log.loc[idx]=[k,coherence_cv]
  idx+=1
  if coherence_cv>=best_cohe:
    best_cohe=coherence_cv
    best_nt=k
    print(f"--> [BEST] Num Topic: {k}, Coherence:{coherence_cv}")
  else:
    print(f"Num Topic: {k}, Coherence:{coherence_cv}")
  print("\n")
df_log.to_csv(f"{name}_Train_History_ProdLDA_lr={lr}_NE={ne}_act={best_act}_solv={best_solv}.csv")

In [ ]:

df_log.to_csv(f"{name}_Train_History_ProdLDA_lr={lr}_NE={ne}_act={best_act}_solv={best_solv}.csv", index=False)

In [ ]:
print(best_nt, best_cohe)

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
avitm = myProdLDA(activation=best_act, solver=best_solv,num_topics=best_nt,batch_size=128,num_epochs=500, lr=3e-3)
avitm.partitioning()
trained_model=avitm.train_model(dataset)
metric = Coherence(texts=dataset.get_corpus(), measure='c_v',topk=20) # Initialize metric
coherence_cv = metric.score(trained_model)
print(coherence_cv)

In [ ]:
w2v=[]
f=open(f"{name}_ProdLDA.txt","w")
for t in trained_model['topics']:
  topics_words.append(t)
  print(len(t),t)
  print(t,file=f)
f.close()

# TOPIC SIMILARITY

In [ ]:
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import gensim.downloader as api
import gensim.downloader
import matplotlib.gridspec as gridspec
import matplotlib
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.tree import export_graphviz
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
import graphviz

In [ ]:
'''
topics_words=[]
def plot_word_cloud(b, ax, v, n):
    topk=10
    sorted_, indices = torch.sort(b, descending=True)
    df = pd.DataFrame(indices[:topk].numpy(), columns=['index']) #PRENDO PRIME "topk" PAROLE
    #print(f"DF: {df}")
    #print(f"VOCAB: {vocab}")
    #print("DEBUG:")
    #print(df.values)
    #for el in df.values:
      #print(el)
    #  print(f"\tk: {el[0]} - v: {diz[el[0]]}")
    words = pd.merge(df, vocab[['index', 'word']], how='left', on='index')['word'].values.tolist()
    #print(f"WORDS: {words}")
    sizes = (sorted_[:topk] * 1000).int().numpy().tolist()
    freqs = {words[i]: sizes[i] for i in range(len(words))}
    #print(f"FREQS: {freqs}\n")
    print(freqs)
    topics_words.append(freqs.keys())
    wc = WordCloud(background_color="white", width=800, height=500)
    wc = wc.generate_from_frequencies(freqs)

    ax.set_title('Topic %d' % (n))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis("off")

if not smoke_test:
    
    beta = prodLDA.beta()
    fig, axs = plt.subplots(2, 5, figsize=(30, 10))
    print(beta.shape[0]) #20
    for n in range(beta.shape[0]):
        i, j = divmod(n, 5)
        #print("\n")
        #print(f"TOPIC: {n}")
        #print(beta[n]) #PESI PER IL TOPIC n
        #print(beta[n].shape) #TANTI PESI QUANTI SONO I MIEI VOCABOLI
        plot_word_cloud(beta[n], axs[i, j], vocab, n)
        #break
    axs[-1, -1].axis('off');

    plt.show()
'''

'\ntopics_words=[]\ndef plot_word_cloud(b, ax, v, n):\n    topk=10\n    sorted_, indices = torch.sort(b, descending=True)\n    df = pd.DataFrame(indices[:topk].numpy(), columns=[\'index\']) #PRENDO PRIME "topk" PAROLE\n    #print(f"DF: {df}")\n    #print(f"VOCAB: {vocab}")\n    #print("DEBUG:")\n    #print(df.values)\n    #for el in df.values:\n      #print(el)\n    #  print(f"\tk: {el[0]} - v: {diz[el[0]]}")\n    words = pd.merge(df, vocab[[\'index\', \'word\']], how=\'left\', on=\'index\')[\'word\'].values.tolist()\n    #print(f"WORDS: {words}")\n    sizes = (sorted_[:topk] * 1000).int().numpy().tolist()\n    freqs = {words[i]: sizes[i] for i in range(len(words))}\n    #print(f"FREQS: {freqs}\n")\n    print(freqs)\n    topics_words.append(freqs.keys())\n    wc = WordCloud(background_color="white", width=800, height=500)\n    wc = wc.generate_from_frequencies(freqs)\n\n    ax.set_title(\'Topic %d\' % (n))\n    ax.imshow(wc, interpolation=\'bilinear\')\n    ax.axis("off")\n\nif not smo

In [ ]:
model_g= api.load('word2vec-google-news-300')

In [ ]:
model = Word2Vec(min_count=1, vector_size=300)
model.build_vocab(w2v)
model.wv.vectors_lockf = np.ones(len(model.wv))
training_examples_count = model.corpus_count
# below line will make it 1, so saving it before
model.build_vocab([list(model_g.key_to_index)], update=True)
model.wv.vectors_lockf = np.ones(len(model.wv))
model.wv.intersect_word2vec_format("GoogleNews-vectors-negative300.bin",binary=True)
model.train(w2v, total_examples=training_examples_count, epochs=model.epochs)


RuntimeError: ignored

In [ ]:
#CLUSTER COMPARISON
tp=w2v
similarities=[]
for i,top_wrd in enumerate(tp):
    print(f"{i+1} - {top_wrd}")
    tmp=tp
    tmp_sim=[]
    for j,other_topic in enumerate(tmp):
        #print(f"\t{j+1} - {other_topic}")
        similarity=model.wv.n_similarity(top_wrd,other_topic)
        #print(f"\t\tSimilarity: {similarity}")
        tmp_sim.append(similarity)
    similarities.append(tmp_sim)
print(similarities)

In [ ]:
with sns.axes_style("darkgrid"):
    mask = np.triu(np.ones_like(similarities, dtype=bool),k=1)
    plt.figure(figsize=(20,20), dpi=80)
    cmap=sns.diverging_palette(240, 10, n=9)
    sns.heatmap(similarities,annot=True,mask=mask,
              cmap=cmap,
              linewidth=2,edgecolor="k",
              #square=True
              vmin=-1,vmax=1,center=0
              )
    plt.title("Similarity between Topics")
    plt.show()

In [ ]:
similarities=pd.DataFrame(similarities)
upper_tri = similarities.where(np.triu(np.ones(similarities.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]
similarities_dropped=similarities.drop(columns=to_drop)
print("Features to be dropped:\n", to_drop)
